<img src="../img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<br><br><br>
<h2><font color="#00586D" size=4>Trabajo Fin de Máster</font></h2>

<h1><font color="#00586D" size=5>Análisis y Predicción de Resultados en Partidas de Clash Royale:<br><b>2. Limpieza y Transformación: Creación del Conjunto de Datos</b></font></h1>
<br><br><br>


<div align="right">
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font><br>
</div>

<font color="#00586D" size=3>Iván Fernández García</font><br>
<font color="#00586D" size=3>Curso académico 2024/2025</font><br>

---

<a id="indice"></a>
<h2><font color="#00586D" size=5>Índice</font></h2>


* [1. Introducción](#section1)
* [2. Creación del conjunto de datos](#section2)
    * [2.1. Limpieza y transformación de cartas](#section2_1)
    * [2.2. Limpieza y transformación de partidas](#section2_2)
* [3. División en entrenamiento y prueba](#section3)
* [4. Conclusiones](#section4)

---

In [2]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

---

<a id="section1"></a>
## <font color="#00586D"> 1. Introducción</font>

En la fase previa del proyecto, se ha recopilado información sobre cartas y partidas a partir de la API oficial de Clash Royale. Una vez disponemos de los datos crudos, es el momento de limpiarlos y aplicar las transformaciones necesarias para crear el conjunto de datos final que utilizaremos para realizar el análisis y el desarrollo de los modelos predictivos.

Para ello, es fundamental convertir registros con estructuras anidadas en datos tabulares de modo que cada fila corresponda a una observación.

---

<a id="section2"></a>
## <font color="#00586D"> 2. Creación del conjunto de datos</font>

A lo largo de esta sección utilizaremos los datos crudos de cartas y partidas de Clash Royale obtenidos en la fase anterior del proyecto. A partir de ellos, aplicaremos las tareas de limpieza y transformación necesarias para crear el conjunto de datos que utilizaremos posteriormente para analizar la información y desarrollar modelos de aprendizaje automático capaces de predecir nuevos enfrentamientos.

<a id="section2_1"></a>
### <font color="#00586D"> 2.1. Limpieza y transformación de cartas</font>

Para las cartas vamos a comenzar leyendo las tropas de coronas, ya que no hay información adicional para ellas. Podemos leer el JSON y normalizarlo. Utilizaremos la misma nomenclatura que la API de Clash Royale para las variables, utilizando además el guion bajo como separador para los atributos de distintos niveles.

In [2]:
with open("../data/raw/support_cards.json", "r", encoding="utf-8") as file:
    support_cards = json.load(file)

df_support_cards = pd.json_normalize(support_cards, sep="_").set_index("id")
df_support_cards.head()

,name,maxLevel,rarity,iconUrls_medium
id,,,,
159000000,Tower Princess,14,common,https://api-assets.clashroyale.com/cards/300/N...
159000001,Cannoneer,9,epic,https://api-assets.clashroyale.com/cards/300/c...
159000002,Dagger Duchess,6,legendary,https://api-assets.clashroyale.com/cards/300/M...
159000004,Royal Chef,6,legendary,https://api-assets.clashroyale.com/cards/300/C...


No necesitamos hacer ninguna modificación, por lo que podemos guardar la tabla:

In [3]:
df_support_cards.to_csv("../data/processed/support_cards.csv", index=True, encoding="utf-8")

Ahora procedemos con las cartas. Primero, vamos a leer la información de la API de la misma forma que en el caso anterior:

In [4]:
with open("../data/raw/cards.json", "r", encoding="utf-8") as file:
    cards_info = json.load(file)

df_cards_info = pd.json_normalize(cards_info, sep="_").set_index("id")
df_cards_info.head()

,name,maxLevel,maxEvolutionLevel,elixirCost,rarity,iconUrls_medium,iconUrls_evolutionMedium
id,,,,,,,
26000000,Knight,14,1.0,3.0,common,https://api-assets.clashroyale.com/cards/300/j...,https://api-assets.clashroyale.com/cardevoluti...
26000001,Archers,14,1.0,3.0,common,https://api-assets.clashroyale.com/cards/300/W...,https://api-assets.clashroyale.com/cardevoluti...
26000002,Goblins,14,NaN,2.0,common,https://api-assets.clashroyale.com/cards/300/X...,NaN
26000003,Giant,12,NaN,5.0,rare,https://api-assets.clashroyale.com/cards/300/A...,NaN
26000004,P.E.K.K.A,9,1.0,7.0,epic,https://api-assets.clashroyale.com/cards/300/M...,https://api-assets.clashroyale.com/cardevoluti...


Las propiedades adicionales tienen un único nivel, por lo que pueden leerse directamente con `pd.read_json()` y `orient="records"` (valor por defecto):

In [5]:
df_card_properties = pd.read_json("../data/raw/card_properties.json").set_index("id")
df_card_properties.head()

,name,winCondition,melee,ranged,air,antiAir,directDamage,splashDamage,resetAttack,type
id,,,,,,,,,,
26000000,Knight,False,True,False,False,False,False,False,False,troop
26000001,Archers,False,False,True,False,True,False,False,False,troop
26000002,Goblins,False,True,False,False,False,False,False,False,troop
26000003,Giant,True,False,False,False,False,False,False,False,troop
26000004,P.E.K.K.A,False,True,False,False,False,False,False,False,troop


Por último, lo mismo con los *counters* pero utilizando `orient="index"`.

In [6]:
df_card_counters = pd.read_json("../data/raw/card_counters.json", orient="index")
df_card_counters.head()

,counters
Knight,"[Knight, Royal Delivery, Ice Golem, Dart Gobli..."
Archers,"[Arrows, Firecracker, Royal Delivery, Skeleton..."
Goblins,"[Fire Spirit, Arrows, Skeleton Dragons, Earthq..."
Giant,"[Cannon, Tesla, Barbarians, Minion Horde, Elit..."
P.E.K.K.A,"[Royal Recruits, Inferno Tower, Witch, P.E.K.K.A]"


A través de los identificadores y los nombres de las cartas, podemos fusionar toda la información en un único *DataFrame* `df_cards`. Además, dado que el nivel máximo de evolución siempre es 1 en caso de que la carta la tenga, transformaremos esta columna para que represente una variable binaria `hasEvolution`.

In [7]:
df_cards = (
    df_cards_info
    .merge(df_card_properties.drop(columns="name"), left_index=True, right_index=True)
    .merge(df_card_counters, left_on="name", right_index=True, how="left")
    .assign(hasEvolution=lambda df: df["maxEvolutionLevel"].notna())
    .drop(columns=["maxEvolutionLevel"])
)
df_cards.head()

,name,maxLevel,elixirCost,rarity,iconUrls_medium,iconUrls_evolutionMedium,winCondition,melee,ranged,air,antiAir,directDamage,splashDamage,resetAttack,type,counters,hasEvolution
id,,,,,,,,,,,,,,,,,
26000000,Knight,14,3.0,common,https://api-assets.clashroyale.com/cards/300/j...,https://api-assets.clashroyale.com/cardevoluti...,False,True,False,False,False,False,False,False,troop,"[Knight, Royal Delivery, Ice Golem, Dart Gobli...",True
26000001,Archers,14,3.0,common,https://api-assets.clashroyale.com/cards/300/W...,https://api-assets.clashroyale.com/cardevoluti...,False,False,True,False,True,False,False,False,troop,"[Arrows, Firecracker, Royal Delivery, Skeleton...",True
26000002,Goblins,14,2.0,common,https://api-assets.clashroyale.com/cards/300/X...,NaN,False,True,False,False,False,False,False,False,troop,"[Fire Spirit, Arrows, Skeleton Dragons, Earthq...",False
26000003,Giant,12,5.0,rare,https://api-assets.clashroyale.com/cards/300/A...,NaN,True,False,False,False,False,False,False,False,troop,"[Cannon, Tesla, Barbarians, Minion Horde, Elit...",False
26000004,P.E.K.K.A,9,7.0,epic,https://api-assets.clashroyale.com/cards/300/M...,https://api-assets.clashroyale.com/cardevoluti...,False,True,False,False,False,False,False,False,troop,"[Royal Recruits, Inferno Tower, Witch, P.E.K.K.A]",True


Podemos cambiar el orden de las columnas para que el resultado tenga un aspecto más limpio:

In [8]:
columns_order = [
    "name", "rarity", "type", "elixirCost", "maxLevel", "hasEvolution",
    "winCondition", "melee", "ranged", "air", "antiAir", "directDamage",
    "splashDamage", "resetAttack", "counters", "iconUrls_medium", "iconUrls_evolutionMedium",
]

df_cards = df_cards[columns_order]
df_cards.head()

,name,rarity,type,elixirCost,maxLevel,hasEvolution,winCondition,melee,ranged,air,antiAir,directDamage,splashDamage,resetAttack,counters,iconUrls_medium,iconUrls_evolutionMedium
id,,,,,,,,,,,,,,,,,
26000000,Knight,common,troop,3.0,14,True,False,True,False,False,False,False,False,False,"[Knight, Royal Delivery, Ice Golem, Dart Gobli...",https://api-assets.clashroyale.com/cards/300/j...,https://api-assets.clashroyale.com/cardevoluti...
26000001,Archers,common,troop,3.0,14,True,False,False,True,False,True,False,False,False,"[Arrows, Firecracker, Royal Delivery, Skeleton...",https://api-assets.clashroyale.com/cards/300/W...,https://api-assets.clashroyale.com/cardevoluti...
26000002,Goblins,common,troop,2.0,14,False,False,True,False,False,False,False,False,False,"[Fire Spirit, Arrows, Skeleton Dragons, Earthq...",https://api-assets.clashroyale.com/cards/300/X...,NaN
26000003,Giant,rare,troop,5.0,12,False,True,False,False,False,False,False,False,False,"[Cannon, Tesla, Barbarians, Minion Horde, Elit...",https://api-assets.clashroyale.com/cards/300/A...,NaN
26000004,P.E.K.K.A,epic,troop,7.0,9,True,False,True,False,False,False,False,False,False,"[Royal Recruits, Inferno Tower, Witch, P.E.K.K.A]",https://api-assets.clashroyale.com/cards/300/M...,https://api-assets.clashroyale.com/cardevoluti...


In [9]:
df_cards.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, 26000000 to 28000024
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      119 non-null    object 
 1   rarity                    119 non-null    object 
 2   type                      119 non-null    object 
 3   elixirCost                118 non-null    float64
 4   maxLevel                  119 non-null    int64  
 5   hasEvolution              119 non-null    bool   
 6   winCondition              119 non-null    bool   
 7   melee                     119 non-null    bool   
 8   ranged                    119 non-null    bool   
 9   air                       119 non-null    bool   
 10  antiAir                   119 non-null    bool   
 11  directDamage              119 non-null    bool   
 12  splashDamage              119 non-null    bool   
 13  resetAttack               119 non-null    bool   
 14  cou

Los valores perdidos se explican a continuación.

* `elixirCost`: Un valor perdido correspondiente a la carta del espejo, cuyo coste depende de la carta con la que se use.
* `counters`: Valores perdidos correspondientes a cartas sin *counters* realmente efectivos, la gran mayoría de ellas hechizos que no hay forma de defender o cartas nuevas.
* `iconUrls_evolutionMedium`: Valores perdidos correspondientes a las cartas sin evolución, que evidentemente tampoco tienen icono de evolución. Esto no ocurre con `hasEvolution`, ya que `maxEvolutionLevel` sí tenía valores perdidos pero después de transformarla en binaria estos valen `False`.

La tabla ya tiene el aspecto deseado, por lo que podemos guardarla.

In [10]:
df_cards.to_csv("../data/processed/cards.csv", index=True, encoding="utf-8")

<a id="section2_2"></a>
### <font color="#00586D"> 2.2. Limpieza y transformación de partidas</font>

Para las partidas, vamos a cargar los datos normalizados utilizando el mismo separador:

In [11]:
with open("../data/raw/battles.json", "r", encoding="utf-8") as file:
    battles = json.load(file)

df_battles = pd.json_normalize(battles, sep="_")
df_battles.head()

,type,battleTime,isLadderTournament,deckSelection,team,opponent,isHostedMatch,leagueNumber,arena_id,arena_name,gameMode_id,gameMode_name
0,PvP,20250414T205256.000Z,False,collection,"[{'tag': '#UJVJ0JJLY', 'name': 'el aña', 'star...","[{'tag': '#8VR2GJL98', 'name': 'Jazzanova Pub'...",False,1,54000020,Legendary Arena,72000006,Ladder
1,PvP,20250404T082559.000Z,False,collection,"[{'tag': '#L9VLRUCR9', 'name': 'elpuercopotter...","[{'tag': '#PJCPRG2L', 'name': 'M3zpkm_511', 's...",False,1,54000019,PANCAKES!,72000006,Ladder
2,PvP,20250412T224452.000Z,False,collection,"[{'tag': '#J99PLVYU', 'name': 'Diechampion', '...","[{'tag': '#LR0GR2VL0', 'name': 'mahp', 'starti...",False,1,54000018,Clash Fest,72000006,Ladder
3,PvP,20250415T161223.000Z,False,collection,"[{'tag': '#JVY2QQ2P2', 'name': 'Kosue.', 'star...","[{'tag': '#VUJURRVL', 'name': '잘생긴멜빙이', 'start...",False,1,54000017,Boot Camp,72000006,Ladder
4,PvP,20250223T150110.000Z,False,collection,"[{'tag': '#8G008GRY', 'name': 'brandon', 'star...","[{'tag': '#20PV89V9LG', 'name': 'BigChunLi', '...",False,1,54000016,Dragon Spa,72000006,Ladder


Vemos que, además de los datos de ambos equipos (en partidas de *Ladder* compuestos por un solo miembro), tenemos otras variables. Muchas de ellas parecen depender del modo de juego, vamos a contar los valores únicos para ver si realmente nos aportan algún tipo de valor.

In [12]:
df_battles.drop(columns=["team", "opponent"]).nunique()

type                      1
battleTime            91766
isLadderTournament        1
deckSelection             1
isHostedMatch             1
leagueNumber              1
arena_id                 23
arena_name               23
gameMode_id               1
gameMode_name             1
dtype: int64

Comprobamos también que no hay valores perdidos:

In [13]:
df_battles.isna().sum()

type                  0
battleTime            0
isLadderTournament    0
deckSelection         0
team                  0
opponent              0
isHostedMatch         0
leagueNumber          0
arena_id              0
arena_name            0
gameMode_id           0
gameMode_name         0
dtype: int64

Se verifica que todos los registros corresponden a partidas 1vs1 del modo de juego *Camino de Trofeos* (*Ladder*). A excepción del *timestamp* de la partida y la información de la arena, el resto de variables toman siempre los valores propios de este tipo de partidas. Por lo tanto, vamos a descartarlas y respecto a la arena nos quedaremos solamente con el nombre y lo renombraremos. Como las listas `team` y `opponent` son siempre equipos de un solo miembro en nuestros datos, podemos utilizar `explode()` sin problema para disponer directamente de los datos de ambos jugadores `player1` y `player2` sin filas adicionales. Por último, transformamos `battleTime` en una variable temporal.

In [14]:
df_battles = (
    df_battles[["battleTime", "arena_name", "team", "opponent"]]
    .explode(["team", "opponent"])
    .rename(columns={"arena_name": "arena", "team": "player1", "opponent": "player2"})
)

df_battles["battleTime"] = pd.to_datetime(df_battles["battleTime"], format="%Y%m%dT%H%M%S.%fZ")
df_battles.head()

,battleTime,arena,player1,player2
0,2025-04-14 20:52:56,Legendary Arena,"{'tag': '#UJVJ0JJLY', 'name': 'el aña', 'start...","{'tag': '#8VR2GJL98', 'name': 'Jazzanova Pub',..."
1,2025-04-04 08:25:59,PANCAKES!,"{'tag': '#L9VLRUCR9', 'name': 'elpuercopotter'...","{'tag': '#PJCPRG2L', 'name': 'M3zpkm_511', 'st..."
2,2025-04-12 22:44:52,Clash Fest,"{'tag': '#J99PLVYU', 'name': 'Diechampion', 's...","{'tag': '#LR0GR2VL0', 'name': 'mahp', 'startin..."
3,2025-04-15 16:12:23,Boot Camp,"{'tag': '#JVY2QQ2P2', 'name': 'Kosue.', 'start...","{'tag': '#VUJURRVL', 'name': '잘생긴멜빙이', 'starti..."
4,2025-02-23 15:01:10,Dragon Spa,"{'tag': '#8G008GRY', 'name': 'brandon', 'start...","{'tag': '#20PV89V9LG', 'name': 'BigChunLi', 's..."


**Importante:** La variable `battleTime` indica el instante en el que se jugó la partida, por lo que debemos tener cuidado para que no se produzca una fuga de datos temporal. Esto no supone un problema siempre que las condiciones dentro del juego sean las mismas (podría considerarse que se han jugado en el mismo periodo), pero sí cuando tenemos **partidas que se han jugado antes y después de un evento que modifica la jugabilidad**. Esto ocurre en los siguientes casos:

* Cambios de balance: No podemos aprender patrones de partidas en las que las cartas tienen determinadas propiedades (por ejemplo, daño) para predecir otras anteriores en las que esta información era diferente.
* Nueva carta: No podemos aprender patrones de partidas en las que se ha utilizado una nueva carta para predecir otras anteriores en las que esta no estaba disponible.

Para evitar tener que gestionar este problema reservando las últimas partidas en el conjunto de prueba y utilizando tipos de validación cruzada específicos para respetar el orden cronológico, nos quedaremos solamente con las partidas que se jugaron después del último de estos eventos (en este caso los cambios de balance del 9 de abril) y consideraremos que todas ellas pertenecen a un mismo periodo donde las condiciones dentro del juego no cambian independientemente del instante temporal de cada una. Idealmente, cada vez que ocurra uno de estos cambios los modelos tendrían que entrenarse con datos posteriores y podrían ser utilizados de manera eficiente hasta que se produzca otro cambio. Los modelos podrían utilizarse incluso de forma indefinida si así se considera (aunque evidentemente podrían ser menos eficientes), pero nunca predecir partidas "pasadas" (anteriores al último de estos eventos) con información "actual" durante validación.

Como desconocemos la hora exacta en la que se aplicaron estos cambios, utilizaremos el día siguiente para asegurarnos de filtrar correctamente:

In [15]:
df_battles = df_battles[df_battles["battleTime"] >= "2025-04-10"]
df_battles.head()

,battleTime,arena,player1,player2
0,2025-04-14 20:52:56,Legendary Arena,"{'tag': '#UJVJ0JJLY', 'name': 'el aña', 'start...","{'tag': '#8VR2GJL98', 'name': 'Jazzanova Pub',..."
2,2025-04-12 22:44:52,Clash Fest,"{'tag': '#J99PLVYU', 'name': 'Diechampion', 's...","{'tag': '#LR0GR2VL0', 'name': 'mahp', 'startin..."
3,2025-04-15 16:12:23,Boot Camp,"{'tag': '#JVY2QQ2P2', 'name': 'Kosue.', 'start...","{'tag': '#VUJURRVL', 'name': '잘생긴멜빙이', 'starti..."
5,2025-04-15 15:48:10,Dragon Spa,"{'tag': '#VY0UGQJ0', 'name': '○●Skual0♌●○', 's...","{'tag': '#Y9CPP889C', 'name': 'KS3', 'starting..."
10,2025-04-15 03:29:55,Silent Sanctuary,"{'tag': '#J22VY9QP0', 'name': 'sebas', 'starti...","{'tag': '#CUJRV2PU', 'name': 'JoCADL!', 'start..."


Vamos a explorar ahora los datos del primer jugador. Para el segundo jugador tendremos lo mismo.

In [16]:
df_player1 = df_battles["player1"].apply(pd.Series)
df_player1.head()

,tag,name,startingTrophies,crowns,kingTowerHitPoints,princessTowersHitPoints,clan,cards,supportCards,globalRank,elixirLeaked,trophyChange
0,#UJVJ0JJLY,el aña,9000.0,1,5844,"[1012, 2825]","{'tag': '#G8Q8UUJV', 'name': 'Guerreros Royal'...","[{'name': 'Firecracker', 'id': 26000064, 'leve...","[{'name': 'Dagger Duchess', 'id': 159000002, '...",None,5.98,NaN
2,#J99PLVYU,Diechampion,8030.0,3,6408,"[4032, 3955]","{'tag': '#G8Q8UUJV', 'name': 'Guerreros Royal'...","[{'name': 'Firecracker', 'id': 26000064, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",None,0.78,29.0
3,#JVY2QQ2P2,Kosue.,7796.0,0,4996,None,"{'tag': '#G8Q8UUJV', 'name': 'Guerreros Royal'...","[{'name': 'Mega Knight', 'id': 26000055, 'leve...","[{'name': 'Dagger Duchess', 'id': 159000002, '...",None,4.60,-29.0
5,#VY0UGQJ0,○●Skual0♌●○,7234.0,0,6408,[291],"{'tag': '#G8Q8UUJV', 'name': 'Guerreros Royal'...","[{'name': 'Barbarians', 'id': 26000008, 'level...","[{'name': 'Tower Princess', 'id': 159000000, '...",None,1.98,-30.0
10,#J22VY9QP0,sebas,6716.0,0,5832,[1524],"{'tag': '#G8Q8UUJV', 'name': 'Guerreros Royal'...","[{'name': 'Knight', 'id': 26000000, 'level': 1...","[{'name': 'Tower Princess', 'id': 159000000, '...",None,1.84,-26.0


In [17]:
df_player1.isna().sum()

tag                            0
name                           0
startingTrophies               7
crowns                         0
kingTowerHitPoints             0
princessTowersHitPoints    25924
clan                         599
cards                          0
supportCards                   0
globalRank                 72783
elixirLeaked                   0
trophyChange                2649
dtype: int64

Vemos que tenemos algunos valores perdidos para `kingTowerHitPoints` (las torres no han recibido daño), `clan` (el jugador no tiene clan), `trophyChange` (por empates u otras razones) y `startingTrophies` (partida inicial). La variable `globalRank` será descartada por su elevada cantidad de valores perdidos que posiblemente se deben a que los jugadores no superan un determinado umbral para entrar en el ranking global o porque nunca existe para partidas de *Ladder*. Debido al número de valores perdidos y al tipo de información que contiene, también se ha decidido eliminar `clan`.

Además, también tenemos que descartar algunas variables de las que no dispondremos en el momento de la predicción, ya que son estadísticas al final de la partida:
* `crowns`: Número de coronas.
* `kingTowerHitPoints`: Daño producido a la torre del rey.
* `kingTowerHitPoints`: Daño producido a las torres de coronas.
* `elixirLeaked`: Elixir malgastado o desaprovechado durante la partida.

Los valores perdidos de las columnas conservadas se gestionarán una vez fusionemos la información de ambos jugadores.

Para el segundo jugador tenemos lo mismo:

In [18]:
df_player2 = df_battles["player2"].apply(pd.Series)
df_player2.head()

,tag,name,startingTrophies,crowns,kingTowerHitPoints,princessTowersHitPoints,clan,cards,supportCards,globalRank,elixirLeaked,trophyChange
0,#8VR2GJL98,Jazzanova Pub,9000.0,0,5624,[3236],"{'tag': '#92PYP9QR', 'name': 'Smoki pl.', 'bad...","[{'name': 'Firecracker', 'id': 26000064, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",None,3.03,NaN
2,#LR0GR2VL0,mahp,8025.0,0,0,None,"{'tag': '#QV0QR9P2', 'name': 'Jackeggs 5G', 'b...","[{'name': 'Mega Knight', 'id': 26000055, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",None,14.06,-25.0
3,#VUJURRVL,잘생긴멜빙이,7802.0,2,6156,"[3909, 1481]","{'tag': '#QQJRVCVJ', 'name': '현조 병의신', 'badgeI...","[{'name': 'Skeletons', 'id': 26000010, 'level'...","[{'name': 'Tower Princess', 'id': 159000000, '...",None,1.50,29.0
5,#Y9CPP889C,KS3,7230.0,1,5748,"[3668, 1607]","{'tag': '#GQGG9', 'name': 'latinos', 'badgeId'...","[{'name': 'Mega Knight', 'id': 26000055, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",None,2.47,30.0
10,#CUJRV2PU,JoCADL!,6752.0,1,5304,"[1237, 3346]",NaN,"[{'name': 'Baby Dragon', 'id': 26000015, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",None,0.00,26.0


In [19]:
df_player2.isna().sum()

tag                            0
name                           0
startingTrophies               5
crowns                         0
kingTowerHitPoints             0
princessTowersHitPoints    22066
clan                       25549
cards                          0
supportCards                   0
globalRank                 72783
elixirLeaked                   0
trophyChange                2828
dtype: int64

Ahora concatenamos los tres *DataFrames*, que comparten id. Añadimos los prefijos `player1` y `player2` para distinguir las columnas.

In [20]:
columns_to_drop = [
    "clan",
    "crowns",
    "kingTowerHitPoints",
    "princessTowersHitPoints",
    "globalRank",
    "elixirLeaked"
]

df_battles = pd.concat(
    [
        df_battles.drop(columns=["player1", "player2"]),
        df_player1.drop(columns=columns_to_drop).add_prefix("player1_"),
        df_player2.drop(columns=columns_to_drop).add_prefix("player2_")
    
    ], axis=1
)

El resultado es el siguiente:

In [21]:
df_battles.head()

,battleTime,arena,player1_tag,player1_name,player1_startingTrophies,player1_cards,player1_supportCards,player1_trophyChange,player2_tag,player2_name,player2_startingTrophies,player2_cards,player2_supportCards,player2_trophyChange
0,2025-04-14 20:52:56,Legendary Arena,#UJVJ0JJLY,el aña,9000.0,"[{'name': 'Firecracker', 'id': 26000064, 'leve...","[{'name': 'Dagger Duchess', 'id': 159000002, '...",NaN,#8VR2GJL98,Jazzanova Pub,9000.0,"[{'name': 'Firecracker', 'id': 26000064, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",NaN
2,2025-04-12 22:44:52,Clash Fest,#J99PLVYU,Diechampion,8030.0,"[{'name': 'Firecracker', 'id': 26000064, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",29.0,#LR0GR2VL0,mahp,8025.0,"[{'name': 'Mega Knight', 'id': 26000055, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",-25.0
3,2025-04-15 16:12:23,Boot Camp,#JVY2QQ2P2,Kosue.,7796.0,"[{'name': 'Mega Knight', 'id': 26000055, 'leve...","[{'name': 'Dagger Duchess', 'id': 159000002, '...",-29.0,#VUJURRVL,잘생긴멜빙이,7802.0,"[{'name': 'Skeletons', 'id': 26000010, 'level'...","[{'name': 'Tower Princess', 'id': 159000000, '...",29.0
5,2025-04-15 15:48:10,Dragon Spa,#VY0UGQJ0,○●Skual0♌●○,7234.0,"[{'name': 'Barbarians', 'id': 26000008, 'level...","[{'name': 'Tower Princess', 'id': 159000000, '...",-30.0,#Y9CPP889C,KS3,7230.0,"[{'name': 'Mega Knight', 'id': 26000055, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",30.0
10,2025-04-15 03:29:55,Silent Sanctuary,#J22VY9QP0,sebas,6716.0,"[{'name': 'Knight', 'id': 26000000, 'level': 1...","[{'name': 'Tower Princess', 'id': 159000000, '...",-26.0,#CUJRV2PU,JoCADL!,6752.0,"[{'name': 'Baby Dragon', 'id': 26000015, 'leve...","[{'name': 'Tower Princess', 'id': 159000000, '...",26.0


Vamos a crear nuestra variable objetivo `winner` a partir de la variable `trophyChange` de ambos jugadores (que será descartada), estableciendo como ganador el jugador con el valor positivo. Nos enfrentamos a un problema de clasificación binaria en el que nuestro objetivo es predecir nuevos enfrentamientos entre dos jugadores cualesquiera, por lo que utilizaremos dos etiquetas `player1` y `player2` en lugar de los *tags* de los ganadores. Por lo tanto, debemos quedarnos solamente con las partidas que tengan un ganador, descartando empates y valores perdidos que se deban a cualquier otra causa. Además, como sabemos que sólo se utiliza una tropa de coronas, podemos utilizar `explode` sin preocupaciones para acceder directamente al único elemento de `SupportCards` (para ambos jugadores).

A modo de curiosidad, hay casos en los que los trofeos que un jugador gana no se corresponden con los que el otro pierde. Esto ocurre porque en ligas bajas no se penalizan tanto las derrotas.

In [22]:
df_battles = (
    df_battles
    .dropna(subset=["player1_trophyChange", "player2_trophyChange", "player1_startingTrophies", "player2_startingTrophies"])
    .explode(["player1_supportCards", "player2_supportCards"])
    .reset_index(drop=True)
    .assign(winner=lambda df: np.where(df["player1_trophyChange"] > df["player2_trophyChange"], "player1", "player2"))
    .drop(columns=["player1_trophyChange", "player2_trophyChange"])
)

df_battles.head()

,battleTime,arena,player1_tag,player1_name,player1_startingTrophies,player1_cards,player1_supportCards,player2_tag,player2_name,player2_startingTrophies,player2_cards,player2_supportCards,winner
0,2025-04-12 22:44:52,Clash Fest,#J99PLVYU,Diechampion,8030.0,"[{'name': 'Firecracker', 'id': 26000064, 'leve...","{'name': 'Tower Princess', 'id': 159000000, 'l...",#LR0GR2VL0,mahp,8025.0,"[{'name': 'Mega Knight', 'id': 26000055, 'leve...","{'name': 'Tower Princess', 'id': 159000000, 'l...",player1
1,2025-04-15 16:12:23,Boot Camp,#JVY2QQ2P2,Kosue.,7796.0,"[{'name': 'Mega Knight', 'id': 26000055, 'leve...","{'name': 'Dagger Duchess', 'id': 159000002, 'l...",#VUJURRVL,잘생긴멜빙이,7802.0,"[{'name': 'Skeletons', 'id': 26000010, 'level'...","{'name': 'Tower Princess', 'id': 159000000, 'l...",player2
2,2025-04-15 15:48:10,Dragon Spa,#VY0UGQJ0,○●Skual0♌●○,7234.0,"[{'name': 'Barbarians', 'id': 26000008, 'level...","{'name': 'Tower Princess', 'id': 159000000, 'l...",#Y9CPP889C,KS3,7230.0,"[{'name': 'Mega Knight', 'id': 26000055, 'leve...","{'name': 'Tower Princess', 'id': 159000000, 'l...",player2
3,2025-04-15 03:29:55,Silent Sanctuary,#J22VY9QP0,sebas,6716.0,"[{'name': 'Knight', 'id': 26000000, 'level': 1...","{'name': 'Tower Princess', 'id': 159000000, 'l...",#CUJRV2PU,JoCADL!,6752.0,"[{'name': 'Baby Dragon', 'id': 26000015, 'leve...","{'name': 'Tower Princess', 'id': 159000000, 'l...",player2
4,2025-04-14 03:26:27,Royal Crypt,#YLGULQU80,Jorge21,6245.0,"[{'name': 'Skeleton Army', 'id': 26000012, 'le...","{'name': 'Tower Princess', 'id': 159000000, 'l...",#VLUJYJGVQ,Spark_Coffee,6246.0,"[{'name': 'Skeletons', 'id': 26000010, 'level'...","{'name': 'Dagger Duchess', 'id': 159000002, 'l...",player1


Ahora vamos a explorar los atributos de la carta de soporte para tener una variable por cada uno de los que sean relevantes.

In [23]:
df_player1_support = df_battles["player1_supportCards"].apply(pd.Series)
df_player1_support.head()

,name,id,level,maxLevel,rarity,iconUrls
0,Tower Princess,159000000,14,14,common,{'medium': 'https://api-assets.clashroyale.com...
1,Dagger Duchess,159000002,5,6,legendary,{'medium': 'https://api-assets.clashroyale.com...
2,Tower Princess,159000000,14,14,common,{'medium': 'https://api-assets.clashroyale.com...
3,Tower Princess,159000000,13,14,common,{'medium': 'https://api-assets.clashroyale.com...
4,Tower Princess,159000000,11,14,common,{'medium': 'https://api-assets.clashroyale.com...


In [24]:
df_player2_support = df_battles["player2_supportCards"].apply(pd.Series)
df_player2_support.head()

,name,id,level,maxLevel,rarity,iconUrls
0,Tower Princess,159000000,14,14,common,{'medium': 'https://api-assets.clashroyale.com...
1,Tower Princess,159000000,14,14,common,{'medium': 'https://api-assets.clashroyale.com...
2,Tower Princess,159000000,13,14,common,{'medium': 'https://api-assets.clashroyale.com...
3,Tower Princess,159000000,12,14,common,{'medium': 'https://api-assets.clashroyale.com...
4,Dagger Duchess,159000002,4,6,legendary,{'medium': 'https://api-assets.clashroyale.com...


Descartaremos los iconos y los niveles máximos.

In [25]:
df_battles = pd.concat(
    [
        df_battles.drop(columns=["player1_supportCards", "player2_supportCards"]),
        df_player1_support[["name", "level", "rarity"]].add_prefix("player1_supportCard_"),
        df_player2_support[["name", "level", "rarity"]].add_prefix("player2_supportCard_")
    
    ], axis=1
)

df_battles.head()

,battleTime,arena,player1_tag,player1_name,player1_startingTrophies,player1_cards,player2_tag,player2_name,player2_startingTrophies,player2_cards,winner,player1_supportCard_name,player1_supportCard_level,player1_supportCard_rarity,player2_supportCard_name,player2_supportCard_level,player2_supportCard_rarity
0,2025-04-12 22:44:52,Clash Fest,#J99PLVYU,Diechampion,8030.0,"[{'name': 'Firecracker', 'id': 26000064, 'leve...",#LR0GR2VL0,mahp,8025.0,"[{'name': 'Mega Knight', 'id': 26000055, 'leve...",player1,Tower Princess,14,common,Tower Princess,14,common
1,2025-04-15 16:12:23,Boot Camp,#JVY2QQ2P2,Kosue.,7796.0,"[{'name': 'Mega Knight', 'id': 26000055, 'leve...",#VUJURRVL,잘생긴멜빙이,7802.0,"[{'name': 'Skeletons', 'id': 26000010, 'level'...",player2,Dagger Duchess,5,legendary,Tower Princess,14,common
2,2025-04-15 15:48:10,Dragon Spa,#VY0UGQJ0,○●Skual0♌●○,7234.0,"[{'name': 'Barbarians', 'id': 26000008, 'level...",#Y9CPP889C,KS3,7230.0,"[{'name': 'Mega Knight', 'id': 26000055, 'leve...",player2,Tower Princess,14,common,Tower Princess,13,common
3,2025-04-15 03:29:55,Silent Sanctuary,#J22VY9QP0,sebas,6716.0,"[{'name': 'Knight', 'id': 26000000, 'level': 1...",#CUJRV2PU,JoCADL!,6752.0,"[{'name': 'Baby Dragon', 'id': 26000015, 'leve...",player2,Tower Princess,13,common,Tower Princess,12,common
4,2025-04-14 03:26:27,Royal Crypt,#YLGULQU80,Jorge21,6245.0,"[{'name': 'Skeleton Army', 'id': 26000012, 'le...",#VLUJYJGVQ,Spark_Coffee,6246.0,"[{'name': 'Skeletons', 'id': 26000010, 'level'...",player1,Tower Princess,11,common,Dagger Duchess,4,legendary


Haremos lo mismo con las cartas del mazo. Como se trata de una lista, accederemos al primer registro para ver la información de cada carta y descartar la que no nos interese.

In [26]:
df_battles.iloc[0]["player1_cards"][0]

{'name': 'Firecracker',
 'id': 26000064,
 'level': 14,
 'evolutionLevel': 1,
 'maxLevel': 14,
 'maxEvolutionLevel': 1,
 'rarity': 'common',
 'elixirCost': 3,
 'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/c1rL3LO1U2D9-TkeFfAC18gP3AO8ztSwrcHMZplwL2Q.png',
  'evolutionMedium': 'https://api-assets.clashroyale.com/cardevolutions/300/c1rL3LO1U2D9-TkeFfAC18gP3AO8ztSwrcHMZplwL2Q.png'}}

En este caso nos quedaremos con todos los atributos menos los iconos y los niveles máximos de carta y evolución. Además utilizaremos algunas de las otras propiedades de las cartas  y transformaremos `evolutionLevel` en `isEvolution` para determinar si el jugador dispone de la evolución de la carta que está utilizando (no confundir con `hasEvolution` en `df_cards`, que indica si una carta tiene o no evolución).

Para ello, podemos utilizar `explode` y obtenemos una carta por fila. Como posteriormente necesitamos volver a juntar las ocho cartas del mazo, pivotaremos la tabla.

In [27]:
df_battles_exploded = df_battles.explode(["player1_cards", "player2_cards"]).reset_index()
df_player1_cards = pd.json_normalize(df_battles_exploded["player1_cards"], sep="_")
df_player1_cards.head()

,name,id,level,evolutionLevel,maxLevel,maxEvolutionLevel,rarity,elixirCost,iconUrls_medium,iconUrls_evolutionMedium,starLevel
0,Firecracker,26000064,14,1.0,14,1.0,common,3.0,https://api-assets.clashroyale.com/cards/300/c...,https://api-assets.clashroyale.com/cardevoluti...,NaN
1,Skeletons,26000010,14,1.0,14,1.0,common,1.0,https://api-assets.clashroyale.com/cards/300/o...,https://api-assets.clashroyale.com/cardevoluti...,NaN
2,Dart Goblin,26000040,12,NaN,12,1.0,rare,3.0,https://api-assets.clashroyale.com/cards/300/B...,https://api-assets.clashroyale.com/cardevoluti...,2.0
3,Rage,28000002,8,NaN,9,NaN,epic,2.0,https://api-assets.clashroyale.com/cards/300/b...,NaN,1.0
4,Giant Skeleton,26000020,10,NaN,9,NaN,epic,6.0,https://api-assets.clashroyale.com/cards/300/0...,NaN,3.0


Descartamos las columnas mencionadas y fusionamos con `df_cards` para añadir otras propiedades relevantes. No hace falta imputar los valores perdidos de `starLevel` y `level` porque se realizarán agregaciones más adelante.

In [28]:
df_player1_cards = (
    df_player1_cards[["name", "level", "starLevel", "evolutionLevel", "rarity", "elixirCost"]]
    .assign(isEvolution=lambda df: df["evolutionLevel"].notna())
    .drop(columns=["evolutionLevel"])
    .merge(df_cards[["name", "type", "winCondition", "melee", "ranged", "air", "antiAir", "directDamage", "splashDamage", "resetAttack"]], left_on="name", right_on="name", how="left")
    .add_prefix("player1_card_")
)

df_player1_cards.head()

,player1_card_name,player1_card_level,player1_card_starLevel,player1_card_rarity,player1_card_elixirCost,player1_card_isEvolution,player1_card_type,player1_card_winCondition,player1_card_melee,player1_card_ranged,player1_card_air,player1_card_antiAir,player1_card_directDamage,player1_card_splashDamage,player1_card_resetAttack
0,Firecracker,14,NaN,common,3.0,True,troop,False,False,True,False,True,False,True,False
1,Skeletons,14,NaN,common,1.0,True,troop,False,True,False,False,False,False,False,False
2,Dart Goblin,12,2.0,rare,3.0,False,troop,False,False,True,False,True,False,False,False
3,Rage,8,1.0,epic,2.0,False,spell,False,False,False,False,True,True,False,False
4,Giant Skeleton,10,3.0,epic,6.0,False,troop,False,True,False,False,False,False,False,False


Repetimos el proceso para el segundo jugador:

In [29]:
df_player2_cards = pd.json_normalize(df_battles_exploded["player2_cards"], sep="_")

df_player2_cards = (
    df_player2_cards[["name", "level", "starLevel", "evolutionLevel", "rarity", "elixirCost"]]
    .assign(isEvolution=lambda df: df["evolutionLevel"].notna())
    .drop(columns=["evolutionLevel"])
    .merge(df_cards[["name", "type", "winCondition", "melee", "ranged", "air", "antiAir", "directDamage", "splashDamage", "resetAttack"]], left_on="name", right_on="name", how="left")
    .fillna(0)
    .add_prefix("player2_card_")
)

df_player2_cards.head()

,player2_card_name,player2_card_level,player2_card_starLevel,player2_card_rarity,player2_card_elixirCost,player2_card_isEvolution,player2_card_type,player2_card_winCondition,player2_card_melee,player2_card_ranged,player2_card_air,player2_card_antiAir,player2_card_directDamage,player2_card_splashDamage,player2_card_resetAttack
0,Mega Knight,7,2.0,legendary,7.0,True,troop,False,True,False,False,False,False,True,False
1,Tesla,15,2.0,common,4.0,True,building,False,False,False,False,True,False,False,False
2,Firecracker,15,1.0,common,3.0,False,troop,False,False,True,False,True,False,True,False
3,Zap,14,0.0,common,2.0,False,spell,False,False,False,False,True,True,True,True
4,Valkyrie,13,2.0,rare,4.0,False,troop,False,True,False,False,False,False,True,False


Concatenamos con el *DataFrame* explotado de modo que cada fila representa una carta de cada jugador y tiene también la información de la partida. Ahora mismo la información de cada partida aparece repetida una vez por cada carta del mazo y podemos distinguir la partida a través de `index`, obtenido después de utilizar `explode()` y `reset_index(drop=False)`.

In [30]:
df_battles_exploded = pd.concat([df_battles_exploded.drop(columns=["player1_cards", "player2_cards"]), df_player1_cards, df_player2_cards], axis=1)
df_battles_exploded.head()

,index,battleTime,arena,player1_tag,player1_name,player1_startingTrophies,player2_tag,player2_name,player2_startingTrophies,winner,...,player2_card_isEvolution,player2_card_type,player2_card_winCondition,player2_card_melee,player2_card_ranged,player2_card_air,player2_card_antiAir,player2_card_directDamage,player2_card_splashDamage,player2_card_resetAttack
0,0,2025-04-12 22:44:52,Clash Fest,#J99PLVYU,Diechampion,8030.0,#LR0GR2VL0,mahp,8025.0,player1,...,True,troop,False,True,False,False,False,False,True,False
1,0,2025-04-12 22:44:52,Clash Fest,#J99PLVYU,Diechampion,8030.0,#LR0GR2VL0,mahp,8025.0,player1,...,True,building,False,False,False,False,True,False,False,False
2,0,2025-04-12 22:44:52,Clash Fest,#J99PLVYU,Diechampion,8030.0,#LR0GR2VL0,mahp,8025.0,player1,...,False,troop,False,False,True,False,True,False,True,False
3,0,2025-04-12 22:44:52,Clash Fest,#J99PLVYU,Diechampion,8030.0,#LR0GR2VL0,mahp,8025.0,player1,...,False,spell,False,False,False,False,True,True,True,True
4,0,2025-04-12 22:44:52,Clash Fest,#J99PLVYU,Diechampion,8030.0,#LR0GR2VL0,mahp,8025.0,player1,...,False,troop,False,True,False,False,False,False,True,False


Para volver a la estructura anterior con toda la información de las cartas, podemos lograrlo haciendo un uso correcto de `pivot()`:

In [31]:
cols_to_pivot = [col for col in df_battles_exploded.columns if col.startswith("player1_card_") or col.startswith("player2_card_")]

index_cols = ["battleTime", "arena", "player1_tag", "player1_name", "player1_startingTrophies", "player2_tag",
              "player2_name", "player2_startingTrophies", "player1_supportCard_name", "player1_supportCard_level",
              "player1_supportCard_rarity", "player2_supportCard_name", "player2_supportCard_level", "player2_supportCard_rarity", "winner"]

df_battles_exploded["card_idx"] = df_battles_exploded.groupby("index").cumcount()
df_battles = df_battles_exploded.pivot(index=index_cols, columns="card_idx", values=cols_to_pivot)

df_battles.columns = [f"{col[0].split("_")[0]}_card{col[1] + 1}_{col[0].split("_")[2]}" for col in df_battles.columns]
df_battles = df_battles.reset_index()
df_battles.head()

,battleTime,arena,player1_tag,player1_name,player1_startingTrophies,player2_tag,player2_name,player2_startingTrophies,player1_supportCard_name,player1_supportCard_level,...,player2_card7_splashDamage,player2_card8_splashDamage,player2_card1_resetAttack,player2_card2_resetAttack,player2_card3_resetAttack,player2_card4_resetAttack,player2_card5_resetAttack,player2_card6_resetAttack,player2_card7_resetAttack,player2_card8_resetAttack
0,2025-04-10 00:01:00,Clash Fest,#GP9GVGLPQ,TALAMBA.507,8030.0,#P2CPJYYYQ,Doom,8030.0,Tower Princess,14,...,False,True,False,False,False,False,False,False,False,False
1,2025-04-10 00:02:12,Silent Sanctuary,#RRCCYJVVV,Big Frost,6701.0,#VCQR8VRV8,Antonio CPC,6693.0,Tower Princess,14,...,False,False,False,False,False,False,False,False,False,False
2,2025-04-10 00:02:14,Royal Crypt,#2L8GV90VQ,RealJuanzito,6371.0,#GVCPVCGRG,angy,6363.0,Tower Princess,13,...,False,True,False,False,False,True,False,True,False,False
3,2025-04-10 00:05:15,Royal Crypt,#VRL2JJUYJ,XaNi,6478.0,#UVGU2L89Y,SmolBruh¤,6488.0,Tower Princess,13,...,False,False,False,True,False,False,False,False,False,False
4,2025-04-10 00:05:31,Clash Fest,#PGJL2YCL0,Eron,8030.0,#LRGLP9CJP,Victor,8030.0,Tower Princess,14,...,True,True,False,False,False,False,False,False,False,True


Llegados a este punto, cada fila representa una observación con toda la información de la partida, incluyendo las propiedades de las cartas que componen los mazos de ambos jugadores.

Sin embargo, debemos hacer algunas consideraciones adicionales:

* En Clash Royale, el mazo se debe ver como un conjunto de cartas. Al aprender de los datos, dos partidas con los mismos mazos y distinto orden para las cartas deberían ser equivalentes.
* Esto también aplica al resto de propiedades de las cartas, es más conveniente realizar agregaciones para describir el mazo.
* Lo anterior no es necesario para las cartas de soporte porque cada jugador utiliza solo una y se puede mantener como categórica.
* Debemos encontrar la forma de incluir información sobre los *counters* en cada observación, lo cual es posible a través de agregaciones.

Además, dentro del juego el nivel (y nivel máximo) de las cartas no difiere entre rarezas, aunque sí en la información devuelta por la API. Podemos aplicar este *bonus* a nuestro conjunto de datos para que las agregaciones que realizaremos posteriormente por nivel de cartas sean más precisas.

In [32]:
rarity_level_bonus = {
    "common": 0,
    "rare": 2,
    "epic": 5,
    "legendary": 8,
    "champion": 10
}

for col in df_battles.columns:
    if col.endswith("_level"):
        rarity_col = col.replace("_level", "_rarity")
        if rarity_col in df_battles.columns:
            df_battles[col] += df_battles[rarity_col].map(rarity_level_bonus)

Ahora, vamos a sustituir las propiedades de las cartas de cada jugador (nivel, rareza, tipo...) por diferentes agregaciones que de cierto modo "describan" los mazos. Esto nos permite representar la información de la partida de una mejor manera.

Concretamente, construiremos las siguientes variables para cada jugador:

* `meanCardLevel`: Nivel medio de las cartas del mazo.
* `minCardLevel`: Nivel mínimo de las cartas del mazo.
* `maxCardLevel`: Nivel máximo de las cartas del mazo.
* `totalStarLevel`: Nivel estelar total de las cartas del mazo (agregación por la suma). Los valores perdidos se ignoran, es como sumar cero.
* `meanElixirCost`: Coste de elixir del mazo (agregación por la media). Para los mazos con espejo, los valores perdidos se ignoran y es como considerar siete cartas.
* `numEvolutionCards`: Número de cartas con evolución en el mazo (el jugador debe tener la evolución, no basta con que la carta tenga evolución).
* `numWinConditionCards`: Número de cartas consideradas *Win Condition* en el mazo.
* `numMeleeCards`: Número de cartas cuerpo a cuerpo en el mazo.
* `numRangedCards`: Número de cartas a distancia en el mazo.
* `numAirCards`: Número de unidades aéreas en el mazo.
* `numAntiAirCards`: Número de cartas con daño a unidades aéreas en el mazo.
* `numDirectDamageCards`: Número de cartas con daño directo a torre en el mazo.
* `numSplashDamageCards`: Número de cartas con daño de salpicadura en el mazo.
* `numResetAttackCards`: Número de cartas con reseteo de ataque en el mazo.
* `numCommonCards`: Número de cartas comunes en el mazo (`rarity="common"`).
* `numRareCards`: Número de cartas raras en el mazo (`rarity="rare"`).
* `numEpicCards`: Número de cartas épicas en el mazo (`rarity="epic"`).
* `numLegendaryCards`: Número de cartas legendarias en el mazo (`rarity="legendary"`).
* `numChampionCards`: Número de campeones en el mazo (`rarity="champion"`).
* `numTroopCards`: Número de tropas en el mazo (`type="troop"`).
* `numBuildingCards`: Número de edificios en el mazo (`type="building"`).
* `numSpellCards`: Número de hechizos en el mazo (`type="spell"`).

In [33]:
df_battles = (
    df_battles.assign(
        player1_meanCardLevel=lambda df: df[[f"player1_card{i}_level" for i in range(1, 9)]].mean(axis=1),
        player2_meanCardLevel=lambda df: df[[f"player2_card{i}_level" for i in range(1, 9)]].mean(axis=1),
        player1_minCardLevel=lambda df: df[[f"player1_card{i}_level" for i in range(1, 9)]].min(axis=1),
        player2_minCardLevel=lambda df: df[[f"player2_card{i}_level" for i in range(1, 9)]].min(axis=1),
        player1_maxCardLevel=lambda df: df[[f"player1_card{i}_level" for i in range(1, 9)]].max(axis=1),
        player2_maxCardLevel=lambda df: df[[f"player2_card{i}_level" for i in range(1, 9)]].max(axis=1),
        player1_totalStarLevel=lambda df: df[[f"player1_card{i}_starLevel" for i in range(1, 9)]].sum(axis=1),
        player2_totalStarLevel=lambda df: df[[f"player2_card{i}_starLevel" for i in range(1, 9)]].sum(axis=1),
        player1_meanElixirCost=lambda df: df[[f"player1_card{i}_elixirCost" for i in range(1, 9)]].mean(axis=1),
        player2_meanElixirCost=lambda df: df[[f"player2_card{i}_elixirCost" for i in range(1, 9)]].mean(axis=1),
        player1_numEvolutionCards=lambda df: df[[f"player1_card{i}_isEvolution" for i in range(1, 9)]].sum(axis=1),
        player2_numEvolutionCards=lambda df: df[[f"player2_card{i}_isEvolution" for i in range(1, 9)]].sum(axis=1),
        player1_numWinConditionCards=lambda df: df[[f"player1_card{i}_winCondition" for i in range(1, 9)]].sum(axis=1),
        player2_numWinConditionCards=lambda df: df[[f"player2_card{i}_winCondition" for i in range(1, 9)]].sum(axis=1),
        player1_numMeleeCards=lambda df: df[[f"player1_card{i}_melee" for i in range(1, 9)]].sum(axis=1),
        player2_numMeleeCards=lambda df: df[[f"player2_card{i}_melee" for i in range(1, 9)]].sum(axis=1),
        player1_numRangedCards=lambda df: df[[f"player1_card{i}_ranged" for i in range(1, 9)]].sum(axis=1),
        player2_numRangedCards=lambda df: df[[f"player2_card{i}_ranged" for i in range(1, 9)]].sum(axis=1),
        player1_numAirCards=lambda df: df[[f"player1_card{i}_air" for i in range(1, 9)]].sum(axis=1),
        player2_numAirCards=lambda df: df[[f"player2_card{i}_air" for i in range(1, 9)]].sum(axis=1),
        player1_numAntiAirCards=lambda df: df[[f"player1_card{i}_antiAir" for i in range(1, 9)]].sum(axis=1),
        player2_numAntiAirCards=lambda df: df[[f"player2_card{i}_antiAir" for i in range(1, 9)]].sum(axis=1),
        player1_numDirectDamageCards=lambda df: df[[f"player1_card{i}_directDamage" for i in range(1, 9)]].sum(axis=1),
        player2_numDirectDamageCards=lambda df: df[[f"player2_card{i}_directDamage" for i in range(1, 9)]].sum(axis=1),
        player1_numSplashDamageCards=lambda df: df[[f"player1_card{i}_splashDamage" for i in range(1, 9)]].sum(axis=1),
        player2_numSplashDamageCards=lambda df: df[[f"player2_card{i}_splashDamage" for i in range(1, 9)]].sum(axis=1),
        player1_numResetAttackCards=lambda df: df[[f"player1_card{i}_resetAttack" for i in range(1, 9)]].sum(axis=1),
        player2_numResetAttackCards=lambda df: df[[f"player2_card{i}_resetAttack" for i in range(1, 9)]].sum(axis=1),
        player1_numCommonCards=lambda df: df[[f"player1_card{i}_rarity" for i in range(1, 9)]].eq("common").sum(axis=1),
        player2_numCommonCards=lambda df: df[[f"player2_card{i}_rarity" for i in range(1, 9)]].eq("common").sum(axis=1),
        player1_numRareCards=lambda df: df[[f"player1_card{i}_rarity" for i in range(1, 9)]].eq("rare").sum(axis=1),
        player2_numRareCards=lambda df: df[[f"player2_card{i}_rarity" for i in range(1, 9)]].eq("rare").sum(axis=1),
        player1_numEpicCards=lambda df: df[[f"player1_card{i}_rarity" for i in range(1, 9)]].eq("epic").sum(axis=1),
        player2_numEpicCards=lambda df: df[[f"player2_card{i}_rarity" for i in range(1, 9)]].eq("epic").sum(axis=1),
        player1_numLegendaryCards=lambda df: df[[f"player1_card{i}_rarity" for i in range(1, 9)]].eq("legendary").sum(axis=1),
        player2_numLegendaryCards=lambda df: df[[f"player2_card{i}_rarity" for i in range(1, 9)]].eq("legendary").sum(axis=1),
        player1_numChampionCards=lambda df: df[[f"player1_card{i}_rarity" for i in range(1, 9)]].eq("champion").sum(axis=1),
        player2_numChampionCards=lambda df: df[[f"player2_card{i}_rarity" for i in range(1, 9)]].eq("champion").sum(axis=1),
        player1_numTroopCards=lambda df: df[[f"player1_card{i}_type" for i in range(1, 9)]].eq("troop").sum(axis=1),
        player2_numTroopCards=lambda df: df[[f"player2_card{i}_type" for i in range(1, 9)]].eq("troop").sum(axis=1),
        player1_numBuildingCards=lambda df: df[[f"player1_card{i}_type" for i in range(1, 9)]].eq("building").sum(axis=1),
        player2_numBuildingCards=lambda df: df[[f"player2_card{i}_type" for i in range(1, 9)]].eq("building").sum(axis=1),
        player1_numSpellCards=lambda df: df[[f"player1_card{i}_type" for i in range(1, 9)]].eq("spell").sum(axis=1),
        player2_numSpellCards=lambda df: df[[f"player2_card{i}_type" for i in range(1, 9)]].eq("spell").sum(axis=1)
    ).drop(columns=[
        f"player1_card{i}_level" for i in range(1, 9)] + [f"player2_card{i}_level" for i in range(1, 9)] +
        [f"player1_card{i}_starLevel" for i in range(1, 9)] + [f"player2_card{i}_starLevel" for i in range(1, 9)] +
        [f"player1_card{i}_elixirCost" for i in range(1, 9)] + [f"player2_card{i}_elixirCost" for i in range(1, 9)] +
        [f"player1_card{i}_rarity" for i in range(1, 9)] + [f"player2_card{i}_rarity" for i in range(1, 9)] +
        [f"player1_card{i}_isEvolution" for i in range(1, 9)] + [f"player2_card{i}_isEvolution" for i in range(1, 9)] +
        [f"player1_card{i}_type" for i in range(1, 9)] + [f"player2_card{i}_type" for i in range(1, 9)] +
        [f"player1_card{i}_winCondition" for i in range(1, 9)] + [f"player2_card{i}_winCondition" for i in range(1, 9)] +
        [f"player1_card{i}_melee" for i in range(1, 9)] + [f"player2_card{i}_melee" for i in range(1, 9)] +
        [f"player1_card{i}_ranged" for i in range(1, 9)] + [f"player2_card{i}_ranged" for i in range(1, 9)] +
        [f"player1_card{i}_air" for i in range(1, 9)] + [f"player2_card{i}_air" for i in range(1, 9)] +
        [f"player1_card{i}_antiAir" for i in range(1, 9)] + [f"player2_card{i}_antiAir" for i in range(1, 9)] +
        [f"player1_card{i}_directDamage" for i in range(1, 9)] + [f"player2_card{i}_directDamage" for i in range(1, 9)] +
        [f"player1_card{i}_splashDamage" for i in range(1, 9)] + [f"player2_card{i}_splashDamage" for i in range(1, 9)] +
        [f"player1_card{i}_resetAttack" for i in range(1, 9)] + [f"player2_card{i}_resetAttack" for i in range(1, 9)]
    )
)

df_battles.head()

,battleTime,arena,player1_tag,player1_name,player1_startingTrophies,player2_tag,player2_name,player2_startingTrophies,player1_supportCard_name,player1_supportCard_level,...,player1_numLegendaryCards,player2_numLegendaryCards,player1_numChampionCards,player2_numChampionCards,player1_numTroopCards,player2_numTroopCards,player1_numBuildingCards,player2_numBuildingCards,player1_numSpellCards,player2_numSpellCards
0,2025-04-10 00:01:00,Clash Fest,#GP9GVGLPQ,TALAMBA.507,8030.0,#P2CPJYYYQ,Doom,8030.0,Tower Princess,14,...,0,2,0,0,6,5,1,1,1,2
1,2025-04-10 00:02:12,Silent Sanctuary,#RRCCYJVVV,Big Frost,6701.0,#VCQR8VRV8,Antonio CPC,6693.0,Tower Princess,14,...,0,1,0,0,5,6,2,0,1,2
2,2025-04-10 00:02:14,Royal Crypt,#2L8GV90VQ,RealJuanzito,6371.0,#GVCPVCGRG,angy,6363.0,Tower Princess,13,...,4,1,0,0,7,4,0,1,1,3
3,2025-04-10 00:05:15,Royal Crypt,#VRL2JJUYJ,XaNi,6478.0,#UVGU2L89Y,SmolBruh¤,6488.0,Tower Princess,13,...,1,1,0,0,7,4,0,2,1,2
4,2025-04-10 00:05:31,Clash Fest,#PGJL2YCL0,Eron,8030.0,#LRGLP9CJP,Victor,8030.0,Tower Princess,14,...,1,2,1,0,5,4,1,1,2,3


Para los *counters*, podemos contar el número total de ellos en el mazo rival. Esto es, para cada carta del mazo, obtener sus counters y ver cuántos está usando el oponente. La suma se guarda en `numCounters`.

In [34]:
def count_counters(row, df_counters, player_prefix, opponent_prefix):
    counter_count = 0
    for i in range(1, 9):
        card_name = row[f"{player_prefix}_card{i}_name"]
        if card_name in df_counters.index:
            counters = df_counters.loc[card_name, "counters"]
            opponent_cards = [row[f"{opponent_prefix}_card{j}_name"] for j in range(1, 9)]
            counter_count += sum(1 for card in opponent_cards if card in counters)
    return counter_count

df_counters = df_cards.set_index("name")[["counters"]].dropna()
df_battles["player1_numCounters"] = df_battles.apply(lambda row: count_counters(row, df_counters, "player1", "player2"), axis=1)
df_battles["player2_numCounters"] = df_battles.apply(lambda row: count_counters(row, df_counters, "player2", "player1"), axis=1)

También podemos crear `numUncounteredCards`, que representa el número de cartas del mazo de cada jugador (entre cero y ocho) sin counters en el mazo rival:

In [35]:
def count_uncountered_cards(row, df_counters, player_prefix, opponent_prefix):
    uncountered_count = 0
    for i in range(1, 9):
        card_name = row[f"{player_prefix}_card{i}_name"]
        if card_name in df_counters.index:
            counters = df_counters.loc[card_name, "counters"]
            opponent_cards = [row[f"{opponent_prefix}_card{j}_name"] for j in range(1, 9)]
            if not any(card in counters for card in opponent_cards):
                uncountered_count += 1
        else:
            uncountered_count += 1
    return uncountered_count

df_battles["player1_numUncounteredCards"] = df_battles.apply(lambda row: count_uncountered_cards(row, df_counters, "player1", "player2"), axis=1)
df_battles["player2_numUncounteredCards"] = df_battles.apply(lambda row: count_uncountered_cards(row, df_counters, "player2", "player1"), axis=1)

Una vez hechas todas estas transformaciones, debemos trabajar también con los nombres de las cartas. Para lograr que no importe el orden de estas y ver el mazo de cada jugador como un conjunto, lo que haremos será crear una variable binaria por cada una de las cartas del juego, de modo que haya exactamente ocho unos por jugador.

In [36]:
card_names = list(df_cards["name"])

player1_binary_columns = pd.DataFrame({
    f"player1_has{card}": df_battles[
        [f"player1_card{i}_name" for i in range(1, 9)]
    ].apply(lambda row: card in row.values, axis=1).astype("int")
    for card in card_names
})

player2_binary_columns = pd.DataFrame({
    f"player2_has{card}": df_battles[
        [f"player2_card{i}_name" for i in range(1, 9)]
    ].apply(lambda row: card in row.values, axis=1).astype("int")
    for card in card_names
})

player1_binary_columns.columns = player1_binary_columns.columns.str.replace(" ", "")
player2_binary_columns.columns = player2_binary_columns.columns.str.replace(" ", "")

df_battles = (
    pd.concat([df_battles, player1_binary_columns, player2_binary_columns], axis=1)
    .drop(columns=[f"player1_card{i}_name" for i in range(1, 9)] + [f"player2_card{i}_name" for i in range(1, 9)])
)

df_battles.head()

,battleTime,arena,player1_tag,player1_name,player1_startingTrophies,player2_tag,player2_name,player2_startingTrophies,player1_supportCard_name,player1_supportCard_level,...,player2_hasTheLog,player2_hasTornado,player2_hasClone,player2_hasEarthquake,player2_hasBarbarianBarrel,player2_hasHealSpirit,player2_hasGiantSnowball,player2_hasRoyalDelivery,player2_hasVoid,player2_hasGoblinCurse
0,2025-04-10 00:01:00,Clash Fest,#GP9GVGLPQ,TALAMBA.507,8030.0,#P2CPJYYYQ,Doom,8030.0,Tower Princess,14,...,0,0,0,0,0,0,0,0,0,0
1,2025-04-10 00:02:12,Silent Sanctuary,#RRCCYJVVV,Big Frost,6701.0,#VCQR8VRV8,Antonio CPC,6693.0,Tower Princess,14,...,0,0,0,0,0,0,0,0,0,0
2,2025-04-10 00:02:14,Royal Crypt,#2L8GV90VQ,RealJuanzito,6371.0,#GVCPVCGRG,angy,6363.0,Tower Princess,13,...,0,0,0,0,0,0,0,0,0,0
3,2025-04-10 00:05:15,Royal Crypt,#VRL2JJUYJ,XaNi,6478.0,#UVGU2L89Y,SmolBruh¤,6488.0,Tower Princess,13,...,1,0,0,0,0,0,0,0,0,0
4,2025-04-10 00:05:31,Clash Fest,#PGJL2YCL0,Eron,8030.0,#LRGLP9CJP,Victor,8030.0,Tower Princess,14,...,1,0,0,0,0,0,0,0,0,0


Por último, podemos reordenar las columnas y renombrar las variables de la carta de soporte para que el resultado sea más limpio.

In [37]:
new_order = (
    ["battleTime", "arena"] +
    ["player1_tag", "player1_name", "player1_startingTrophies"] +
    player1_binary_columns.columns.tolist() +
    ["player1_meanCardLevel", "player1_minCardLevel", "player1_maxCardLevel", "player1_totalStarLevel",
     "player1_meanElixirCost", "player1_numEvolutionCards", "player1_numWinConditionCards", "player1_numMeleeCards",
     "player1_numRangedCards", "player1_numAirCards", "player1_numAntiAirCards", "player1_numDirectDamageCards",
     "player1_numSplashDamageCards", "player1_numResetAttackCards", "player1_numCommonCards", "player1_numRareCards",
    "player1_numEpicCards", "player1_numLegendaryCards", "player1_numChampionCards", "player1_numTroopCards",
    "player1_numBuildingCards", "player1_numSpellCards", "player1_numCounters", "player1_numUncounteredCards"] +
    ["player1_supportCard_name", "player1_supportCard_level", "player1_supportCard_rarity"] +
    ["player2_tag", "player2_name", "player2_startingTrophies"] +
    player2_binary_columns.columns.tolist() +
    ["player2_meanCardLevel", "player2_minCardLevel", "player2_maxCardLevel", "player2_totalStarLevel",
     "player2_meanElixirCost", "player2_numEvolutionCards", "player2_numWinConditionCards", "player2_numMeleeCards",
     "player2_numRangedCards", "player2_numAirCards", "player2_numAntiAirCards", "player2_numDirectDamageCards",
     "player2_numSplashDamageCards", "player2_numResetAttackCards", "player2_numCommonCards", "player2_numRareCards",
    "player2_numEpicCards", "player2_numLegendaryCards", "player2_numChampionCards", "player2_numTroopCards",
    "player2_numBuildingCards", "player2_numSpellCards", "player2_numCounters", "player2_numUncounteredCards"] +
    ["player2_supportCard_name", "player2_supportCard_level", "player2_supportCard_rarity"] +
    ["winner"]
)

df_battles = (
    df_battles[new_order]
    .rename(columns={
        "player1_supportCard_name": "player1_supportCardName",
        "player1_supportCard_level": "player1_supportCardLevel",
        "player1_supportCard_rarity": "player1_supportCardRarity",
        "player2_supportCard_name": "player2_supportCardName",
        "player2_supportCard_level": "player2_supportCardLevel",
        "player2_supportCard_rarity": "player2_supportCardRarity"
    })
)

df_battles.head()

,battleTime,arena,player1_tag,player1_name,player1_startingTrophies,player1_hasKnight,player1_hasArchers,player1_hasGoblins,player1_hasGiant,player1_hasP.E.K.K.A,...,player2_numChampionCards,player2_numTroopCards,player2_numBuildingCards,player2_numSpellCards,player2_numCounters,player2_numUncounteredCards,player2_supportCardName,player2_supportCardLevel,player2_supportCardRarity,winner
0,2025-04-10 00:01:00,Clash Fest,#GP9GVGLPQ,TALAMBA.507,8030.0,0,0,0,0,0,...,0,5,1,2,6,4,Tower Princess,14,common,player1
1,2025-04-10 00:02:12,Silent Sanctuary,#RRCCYJVVV,Big Frost,6701.0,1,0,0,0,0,...,0,6,0,2,5,4,Cannoneer,14,epic,player1
2,2025-04-10 00:02:14,Royal Crypt,#2L8GV90VQ,RealJuanzito,6371.0,1,0,0,0,0,...,0,4,1,3,4,4,Tower Princess,13,common,player2
3,2025-04-10 00:05:15,Royal Crypt,#VRL2JJUYJ,XaNi,6478.0,0,0,0,0,0,...,0,4,2,2,7,5,Dagger Duchess,12,legendary,player1
4,2025-04-10 00:05:31,Clash Fest,#PGJL2YCL0,Eron,8030.0,0,0,0,0,0,...,0,4,1,3,9,3,Tower Princess,14,common,player2


Para resolver nuestro problema, lo ideal sería que el conjunto de datos estuviera completamente balanceado. De este modo, si los modelos tienden a predecir uno de los jugadores se deberá únicamente a los patrones detectados, pero nunca al número de ejemplos de cada clase. Vamos a comprobar cuántos tenemos:

In [39]:
class_counts = df_battles["winner"].value_counts()
class_counts

winner
player2    36690
player1    31486
Name: count, dtype: int64

Aunque haya poca diferencia, deberíamos reducir este riesgo en la medida de lo posible para evitar que el resultado se altere en exceso cuando cambiamos el orden de los dos jugadores. Por lo tanto, vamos a aplicar un submuestreo para tener exactamente el mismo número de ejemplos de cada clase:

In [40]:
df_battles_balanced = pd.concat([df_battles[df_battles["winner"] == "player1"], df_battles[df_battles["winner"] == "player2"].sample(class_counts.min(), random_state=42)])
df_battles_balanced["winner"].value_counts()

winner
player1    31486
player2    31486
Name: count, dtype: int64

Podemos guardar nuestro conjunto de datos:

In [ ]:
df_battles_balanced.to_csv("../data/processed/battles.csv", index=False, encoding="utf-8")

---

<a id="section3"></a>
## <font color="#00586D"> 3. División en entrenamiento y prueba</font>

Nuestro conjunto de datos está listo para ser utilizado, por lo que podemos dar inicio a las fases habituales de análisis exploratorio, preprocesamiento y modelado.

Antes de continuar, vamos a dividirlo en un subconjunto de entrenamiento (80%) y otro de prueba (20%). Reservaremos el segundo para la evaluación final de modelos, de modo que la información de sus partidas no se utilice hasta entonces (tampoco durante el análisis exploratorio, pues estaríamos tomando decisiones en base a estos datos).

Podemos realizar una partición estratificada utilizando el método `train_test_split()` de *Scikit-learn*:

In [42]:
target = "winner"
y = df_battles_balanced[target]
X = df_battles_balanced.drop(columns=[target])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

Una vez realizada la partición, uniremos la variable objetivo con el resto de variables predictoras para guardar los datos solamente en dos ficheros. Cuando sea necesario usarlos, volveremos a separar.

In [43]:
pd.concat([X_train, y_train], axis=1).to_csv("../data/final/train.csv", index=False, encoding="utf-8")
pd.concat([X_test, y_test], axis=1).to_csv("../data/final/test.csv", index=False, encoding="utf-8")

---

<a id="section4"></a>
## <font color="#00586D"> 4. Conclusiones</font>

Durante esta fase, se ha realizado una limpieza completa de los datos crudos provenientes de la API, eliminando los registros sin información sobre el resultado y descartando variables irrelevantes para nuestro problema (no cambian para el tipo de partidas con las que estamos trabajando) o que no estarán disponibles a la hora de predecir (estadísticas al final de la partida como las coronas o el daño a las torres).

Se han transformado los datos en un formato estructurado y usable, de modo que cada fila represente una observación que contiene información sobre la partida y sobre los mazos utilizados por ambos jugadores. Esto ha supuesto usar todas las propiedades de las cartas, los *counters*, realizar agregaciones y utilizar codificación binaria para que no importe el orden de las cartas en los mazos.

Cuando partimos directamente de un conjunto de datos para resolver el problema, el proceso de ingeniería de características o *feature engineering* se limita a las fases de análisis exploratorio de datos y preprocesamiento. Sin embargo, en proyectos como este donde es necesario limpiar y transformar los datos crudos, es habitual que también esté presente durante las fases previas para poder construir un *dataset* lo suficientemente valioso. Es lo que ha ocurrido en este caso, si bien es cierto que la creación de nuevos atributos a partir del conjunto de datos resultante también estará presente de aquí en adelante.

Como futura mejora, se podría automatizar este proceso para facilitar la transformación de partidas en crudo obtenidas a través de la API oficial de Clash Royale. Se podrían almacenar los datos en base a diferentes criterios de calidad e implementar *ETLs* utilizando una arquitectura *Medallion* o similar. Además, hay que tener en cuenta que la jugabilidad se modifica con los cambios de balance o el lanzamiento de nuevas cartas, por lo que los datos sobre los que se realicen tareas de *Machine Learning* deben pertenecer a un mismo periodo donde las condiciones del juego no cambien. En caso de no distinguir entre partidas anteriores y posteriores a estos eventos, se deben seguir las pautas adecuadas para no incurrir en fugas de datos temporales.

---